In [27]:
from keras.models import Model
from keras.layers import Dense,Input, Conv1D, GlobalMaxPooling1D, MaxPooling1D,Flatten
import numpy as np
from numpy import argmax
from keras.utils import to_categorical
from keras import backend as K

stride를 3개 처럼 전체로 주면 정확도가 떨어짐  
filter 개수는 자유

Conv 1D

In [4]:
#RNN은 3차원 구조로 맞추어야 한다.
x_train=np.random.random([10, 8, 3]) #8개 단어(단어 1개가 3개 feature로 구성: word Embedding Vector) (리뷰 문서 1개)가 10개
                                       #D0, D1, D2 (면, 행, 열) (10, 8, 3)
                                       #10 대신 none을 써야 fit에서 test의 데이터 개수는 1개기 때문에 들어오는 대로 받아준다.
y_train=np.random.choice([0,1],[10,1])
z_test=np.random.random([1, 8, 3]) 

In [5]:
np.round(x_train,3) #여기 가로 행 줄은 단어 하나를 나타냄(feature)
                    #8개 세로로 나열된 단어는 순차적(시간축, step수)로 나열된다. (RNN 박스 구조 순차)

array([[[0.546, 0.948, 0.533],
        [0.739, 0.177, 0.196],
        [0.948, 0.27 , 0.645],
        [0.195, 0.762, 0.447],
        [0.537, 0.583, 0.46 ],
        [0.185, 0.416, 0.328],
        [0.422, 0.125, 0.907],
        [0.483, 0.948, 0.708]],

       [[0.14 , 0.255, 0.563],
        [0.844, 0.66 , 0.867],
        [0.889, 0.432, 0.058],
        [0.167, 0.867, 0.228],
        [0.003, 0.433, 0.176],
        [0.258, 0.687, 0.473],
        [0.441, 0.93 , 0.19 ],
        [0.786, 0.943, 0.424]],

       [[0.541, 0.397, 0.386],
        [0.32 , 0.112, 0.767],
        [0.359, 0.304, 0.91 ],
        [0.348, 0.033, 0.142],
        [0.976, 0.845, 0.259],
        [0.092, 0.889, 0.385],
        [0.719, 0.879, 0.233],
        [0.448, 0.544, 0.941]],

       [[0.478, 0.481, 0.882],
        [0.321, 0.727, 0.051],
        [0.849, 0.73 , 0.828],
        [0.449, 0.284, 0.718],
        [0.207, 0.896, 0.417],
        [0.533, 0.011, 0.755],
        [0.089, 0.648, 0.306],
        [0.24 , 0.467, 0.137]],



In [105]:
np.round(x_train,3) #각 8개 단어씩 1열은 리뷰 1개를 의미----->RNN 박스는 8개!

array([[[0.663, 0.461, 0.206, 0.132, 0.667],
        [0.649, 0.96 , 0.741, 0.148, 0.736],
        [0.627, 0.27 , 0.363, 0.073, 0.784],
        [0.538, 0.372, 0.357, 0.086, 0.801],
        [0.769, 0.485, 0.541, 0.973, 0.733],
        [0.588, 0.868, 0.821, 0.493, 0.526],
        [0.869, 0.681, 0.845, 0.696, 0.145],
        [0.215, 0.243, 0.308, 0.268, 0.302]],

       [[0.113, 0.745, 0.154, 0.004, 0.017],
        [0.249, 0.268, 0.406, 0.415, 0.808],
        [0.167, 0.924, 0.548, 0.369, 0.202],
        [0.986, 0.065, 0.877, 0.481, 0.364],
        [0.167, 0.623, 0.022, 0.813, 0.891],
        [0.38 , 0.977, 0.375, 0.573, 0.238],
        [0.794, 0.033, 0.851, 0.816, 0.8  ],
        [0.628, 0.435, 0.186, 0.995, 0.798]],

       [[0.828, 1.   , 0.335, 0.543, 0.209],
        [0.25 , 0.292, 0.312, 0.763, 0.519],
        [0.538, 0.52 , 0.329, 0.487, 0.62 ],
        [0.973, 0.6  , 0.371, 0.5  , 0.926],
        [0.336, 0.123, 0.609, 0.096, 0.852],
        [0.212, 0.455, 0.162, 0.838, 0.096],
      

In [106]:
y_train

array([[1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0]])

In [107]:
z_test

array([[[0.83838372, 0.68362611, 0.25371746, 0.8656221 , 0.59611821],
        [0.90931581, 0.11743086, 0.1344557 , 0.65857627, 0.17946454],
        [0.01481831, 0.76909776, 0.00651659, 0.45500092, 0.7257053 ],
        [0.19846867, 0.41574747, 0.17820781, 0.47372872, 0.97376083],
        [0.90785764, 0.83977268, 0.45429221, 0.23950586, 0.94575677],
        [0.59355787, 0.84179764, 0.71768748, 0.6940324 , 0.50478067],
        [0.78904892, 0.70883128, 0.0369805 , 0.06655861, 0.21064579],
        [0.90435637, 0.82212014, 0.12493693, 0.62212914, 0.01160812]]])

In [28]:
K.clear_session()

x_input = Input(batch_shape = (None, 8, 3))
x_Conv =Conv1D(12, 3, activation='relu')(x_input) #3줄로 된 filter를 12개 만들어라
#globalmaxpooling 시키지 않고 maxpooling-> 가장 큰 값을 시키지 않고 3개짜리 필터를 stride 또 시켜줌 ->2D와 유사해짐
x_Pooling=MaxPooling1D(3, strides=1)(x_Conv)
x_Flat=Flatten()(x_Pooling) #Flatten=여러 차원을 1줄로 나열
x_output = Dense(1, activation='sigmoid')(x_Flat)

model=Model(x_input, x_output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8, 3)              0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 6, 12)             120       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4, 12)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 48)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 49        
Total params: 169
Trainable params: 169
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='binary_crossentropy',optimizer='adam')

In [19]:
model.fit(x_train, y_train, epochs=50, batch_size=1)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/50
10/10 [==============================] - 1s 74ms/step - loss: 1.1715
Epoch 2/50
10/10 [==============================] - 0s 1ms/step - loss: 1.0972
Epoch 3/50
10/10 [==============================] - 0s 1ms/step - loss: 1.0243
Epoch 4/50
10/10 [==============================] - 0s 1ms/step - loss: 0.9517
Epoch 5/50
10/10 [==============================] - 0s 900us/step - loss: 0.9029
Epoch 6/50
10/10 [==============================] - 0s 1ms/step - loss: 0.8553
Epoch 7/50
10/10 [==============================] - 0s 1ms/step - loss: 0.8228
Epoch 8/50
10/10 [==============================] - 0s 900us/step - loss: 0.7898
Epoch 9/50
10/10 [==============================] - 0s 1ms/step - loss: 0.7650
Epoch 10/50
10/10 [==============================] - 0s 900us/step - loss: 0.7446
Epoch 11/50
10/10 [==============================] - 0s 900us/step - loss: 0

In [20]:
a=model.predict(z_test, batch_size=1)
a

array([[0.35577607]], dtype=float32)

In [21]:
a.argmax()

0